In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

d:\Mloops Project for practise\YT-Capstone-project\mloops-capstone-project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('IMDB.csv')

In [3]:
df = df.sample(500)
df.to_csv('data.csv' , index= False)

In [4]:
df.head()

,review,sentiment
587,"Okay, so I'm not a big video game buff, but wa...",negative
564,I like this film for several reasons. I have a...,positive
766,"This movie was fun, if all over the board.<br ...",positive
740,So I'm looking to rent a DVD and I come across...,negative
157,There is a reason to call this a teen flick. O...,negative


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df = normalize_text(df)

df.head()

,review,sentiment
587,okay big video game buff game house dead reall...,negative
564,like film several reason soft spot film intric...,positive
766,movie fun board br br it essentially follows c...,positive
740,looking rent dvd come across movie called end ...,negative
157,reason call teen flick possible point would pu...,negative


In [7]:
pd.DataFrame(df['sentiment'].value_counts()).reset_index()

,sentiment,count
0,negative,265
1,positive,235


In [8]:
df.head()

,review,sentiment
587,okay big video game buff game house dead reall...,negative
564,like film several reason soft spot film intric...,positive
766,movie fun board br br it essentially follows c...,positive
740,looking rent dvd come across movie called end ...,negative
157,reason call teen flick possible point would pu...,negative


In [9]:
df = df[df['sentiment'].isin(['positive','negative'])]

In [10]:
df['sentiment'] = df['sentiment'].map({'positive':1 , 'negative':0})

In [11]:
df.head()

,review,sentiment
587,okay big video game buff game house dead reall...,0
564,like film several reason soft spot film intric...,1
766,movie fun board br br it essentially follows c...,1
740,looking rent dvd come across movie called end ...,0
157,reason call teen flick possible point would pu...,0


In [12]:
pd.DataFrame(df['sentiment'].value_counts()).reset_index()

,sentiment,count
0,0,265
1,1,235


In [13]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features = 50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train , X_test , y_train , y_test = train_test_split(X,y , test_size = 0.2 , random_state = 42)

In [16]:
import dagshub

mlflow.set_tracking_uri("https://dagshub.com/nikhilsingh88801/YT-Capstone-project-1.mlflow")
dagshub.init(repo_owner='nikhilsingh88801', repo_name='YT-Capstone-project-1', mlflow=True)

mlflow.set_experiment('Logistic Regression Baseline')

Accessing as nikhilsingh88801

Initialized MLflow to track repo "nikhilsingh88801/YT-Capstone-project-1"

Repository nikhilsingh88801/YT-Capstone-project-1 initialized!

<Experiment: artifact_location='mlflow-artifacts:/d0b2f0fdd08b46eca8f057e1236a7531', creation_time=1771332052362, experiment_id='0', last_update_time=1771332052362, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score

# Configure logging
logging.basicConfig(level= logging.INFO , format= "%(asctime)s - %(levelname)s - %(message)s")
logging.info("Starting Mlflow run...")

with mlflow.start_run(run_name="CountVectorizer version-1"):
    start_time = time.time()

    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)

        logging.info("Fitting the model...")
        model.fit(X_train,y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model","LogisticRegression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model , "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")
    
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    
    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info= True)

2026-02-17 18:55:50,470 - INFO - Starting Mlflow run...
2026-02-17 18:55:51,199 - INFO - Logging preprocessing parameters...
2026-02-17 18:55:52,349 - INFO - Initializing Logistic Regression model...
2026-02-17 18:55:52,349 - INFO - Fitting the model...
2026-02-17 18:55:52,349 - INFO - Model training complete.
2026-02-17 18:55:52,349 - INFO - Logging model parameters...
2026-02-17 18:55:52,745 - INFO - Making predictions...
2026-02-17 18:55:52,745 - INFO - Calculating evaluation metrics...
2026-02-17 18:55:52,753 - INFO - Logging evaluation metrics...
2026-02-17 18:55:54,236 - INFO - Saving and logging the model...
2026/02/17 18:55:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
d:\Mloops Project for practise\YT-Capstone-project\mloops-capstone-project\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's obj

🏃 View run CountVectorizer version-1 at: https://dagshub.com/nikhilsingh88801/YT-Capstone-project-1.mlflow/#/experiments/0/runs/787d8344759d464285b61938085a78ef
🧪 View experiment at: https://dagshub.com/nikhilsingh88801/YT-Capstone-project-1.mlflow/#/experiments/0
